In [135]:
import pymongo
import re
from tika import parser

pdf = parser.from_file('pdfs/premiados_bloco2_2020.pdf')
# pega o texto do pdf
texto = pdf['content']
texto = texto.replace('\n', ' ')
texto = texto.replace('\t', ' ')
texto = " ".join(texto.split())
# padroniza as palavras chaves para splits
texto = texto.replace('As contempladas','Os contemplados')
texto = texto.replace('Os projetos', 'Os contemplados')
texto = texto.replace('Os estudantes contemplados', 'Os contemplados')
texto = texto.replace('[ PROJETO:', '[PROJETO:')
# separa os premios por tipo
texto = texto.split('[Prêmios FEBRACE]')[0]
texto = texto.split('PRÊMIOS DA ORGANIZAÇÃO DA FEBRACE NA VOTAÇÃO POPULAR')[0]
# separa por empresa
texto = texto.split('[EMPRESA/INSTITUIÇÃO]')
texto = texto[1:]

def teste(page):
    empresa = texto[page].split('[PRÊMIO]')
    premio = empresa[1].split('Os contemplados')
    projeto = premio[1].split('[PROJETO:')[1].split(']')
    projetoJunto = "]".join(projeto)
    projeto[1] = projeto[1].replace('–', '-') # mais uma padronizacao
    separator = re.findall('- [A-Z][A-Z] ', projeto[1])
    cidade = projetoJunto.split(separator[0])
    estado = separator[0].split(' ')
    
    alunos = cidade[1].split(" [Estudante] ")
    orientador = alunos.pop()
    orientador = orientador.split('[Orientador(a)]')
    coorientadores = orientador[1]
    
    empresa_ = empresa[0].strip()
    premio_ = premio[0].strip()
    projeto_ = projeto[0].split(']')[0].strip()
    cidade[0] = cidade[0].replace('Do', 'De') # sim cara, mais uma
    cidade_ = cidade[0].split(' De ')[1]# colocar o strip :)
    estado_ = estado[1].strip()

    alunos_ = ', '.join(alunos)
    orientador_ = orientador[0]
    coorientador_ = orientador[1].split('[Coorientador(a)]')[0].strip()
       
    
    return { "empresa": empresa_, "premio": premio_, "projeto": projeto_, "cidade": cidade_, "estado": estado_, "alunos": alunos_, "orientador": orientador_, "coorientador": coorientador_ }


In [136]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["datascience"]
col = db["premiados"]

for i in range(len(texto)):
    try:
        x = col.insert_one(teste(i))
    except Exception as e:
        print(e)

In [123]:
for x in col.find():
    print(x)

{'_id': ObjectId('6083530b01f0050423434a9b'), 'empresa': 'American Psychological Association', 'premio': 'Outstanding for Behavioral Sciences', 'projeto': 'Gravidez e maternidade na adolescência: mobilidade social e sociabilidade local na periferia de São Paulo', 'cidade': 'SÃO PAULO ', 'estado': 'SP', 'alunos': 'Eloisa Maria de Souza Falcão, Maria Luiza de Oliveira Jorge', 'orientador': 'Ednilson Aparecido Quarenta ', 'coorientador': ''}
{'_id': ObjectId('6083532901f0050423434a9c'), 'empresa': 'Intel Excellence in Computer Science', 'premio': 'Intel Excellence in Computer Science', 'projeto': 'Solidari: plataforma que interliga instituições sociais, doadores e voluntários, com o intuito de propagar o voluntariado no Brasil', 'cidade': 'CAMPINAS ', 'estado': 'SP', 'alunos': 'Gabriela Nery Batista, Mayra Fernanda Mendes Braga', 'orientador': 'Sérgio Luiz Moral Marques ', 'coorientador': 'Simone Pierini Facini Rocha'}
{'_id': ObjectId('6083532901f0050423434a9d'), 'empresa': 'RICOH Americ